In [ ]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
from PIL import ImageFont
from matplotlib.font_manager import FontProperties
import time
import json

In [ ]:
class Evaluation:
    def __init__(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true

    def nrmse(self):
        mse = np.mean(np.abs(self.y_true-self.y_pred)**2)
        rmse = np.sqrt(mse)
        return rmse / np.ptp(np.abs(self.y_true))

    def mase(self):
        n = len(self.y_true)
        d = np.abs(np.diff(self.y_true)).sum() / (n - 1)
        errors = np.abs(self.y_true - self.y_pred)
        return errors.mean() / d

    def mape(self):
        return np.mean(np.abs((self.y_true - self.y_pred) / self.y_true)) * 100
    
    def rmse(self):
        mse = np.mean(np.abs(self.y_true-self.y_pred)**2)
        rmse = np.sqrt(mse)
        return rmse
    
    def mae(self):
        errors = np.abs(self.y_true - self.y_pred)
        return errors.mean()
    
    def r2_score(self):
        numerator = np.sum(np.abs(self.y_true - self.y_pred) ** 2)
        denominator = np.sum(np.abs(self.y_true - np.mean(self.y_true)) ** 2)
        return 1 - (numerator / denominator)

In [ ]:
class GeneratePlots:
    def __init__(self, y_test, y_pred):
        self.y_pred = y_pred
        self.y_test = y_test

    def plot_model_evaluation(self):
        # Configuration settings
        linewidth = 2
        markersize = 10
        fontname = 'Times New Roman'
        fontsize = 20
        tick_fontsize = 14
        legend_fontsize = 14
        savefig = 0
        colours = np.array([
            [0.0000, 0.4470, 0.7410],
            [0.8500, 0.3250, 0.0980],
            [0.9290, 0.6940, 0.1250],
            [0.4940, 0.1840, 0.5560],
            [0.4660, 0.6740, 0.1880],
            [0.3010, 0.7450, 0.9330],
            [0.6350, 0.0780, 0.1840],
            [0.0000, 0.0000, 0.0000]
        ])
        
        M = self.y_test.shape[2]  # Number of users
        K = self.y_test.shape[0]
        # 55986,55986+23994
        # 63984, 63984+15996
        time_steps = np.arange(K)

        for user in range(M):
            # Extract the amplitude and phase of the received signals for each user
            amplitude_test = self.y_test[:, :, user, 0]
            phase_test = self.y_test[:, :, user, 1]
            amplitude_pred = self.y_pred[:, :, user, 0]
            phase_pred = self.y_pred[:, :, user, 1]

            # Calculate the mean of phase_pred and scale the deviations from the mean
            mean_phase_pred = np.mean(phase_pred)
            scaled_phase_pred = mean_phase_pred + 2.0 * (phase_pred - mean_phase_pred)

            # Calculate the amplitude of the received signals for each user
            amplitude_test = amplitude_test ** 2
            amplitude_pred = amplitude_pred ** 2

            # Normalize the amplitudes
            normalized_amplitude_test = amplitude_test
            normalized_amplitude_pred = amplitude_pred

            # Plot the normalized received power for each user and SNR
            fig, ax = plt.subplots(figsize=(12, 5))  # Size in inches
            ax.plot(time_steps, normalized_amplitude_test, label='Target', color=colours[0,:], linewidth=linewidth)
            ax.plot(time_steps, normalized_amplitude_pred, label='Predicted', linestyle='--', color=colours[1,:], linewidth=linewidth)
            ax.set_xlabel('Time (in samples)', fontname=fontname, fontsize=fontsize)
            ax.set_ylabel('Cascaded channel magnitude', fontname=fontname, fontsize=fontsize)
            ax.set_xlim([0, 20000])
            ax.set_ylim([0, 20])
            legend_prop = FontProperties(family=fontname, size=legend_fontsize)
            ax.legend(frameon=False, prop=legend_prop)
            ax.grid(False)
            ax.tick_params(axis='x', labelsize=tick_fontsize)
            ax.tick_params(axis='y', labelsize=tick_fontsize)

            # Add vertical line at x=14000
            ax.axvline(x=15996, color='black', linewidth=2)

            # Add annotations for "Train" and "Test"
            ax.annotate('', xy=(14000, 1.9), xytext=(16000, 1.9), fontsize=fontsize,
                        arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=8), ha='center', va='center')
            ax.annotate('', xy=(17000, 1.9), xytext=(16000, 1.9), fontsize=fontsize,
                        arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=8), ha='center', va='center')
            
            # Add annotations for text
            ax.text(14000, 3, 'Train', fontsize=fontsize, ha='center', va='center')
            ax.text(17500, 3, 'Test', fontsize=fontsize, ha='center', va='center')


            # Create an inset plot
            ax_inset = inset_axes(ax, width="30%", height="30%", loc='upper right', borderpad=2)
            ax_inset.plot(time_steps, normalized_amplitude_test, color=colours[0,:], linewidth=linewidth)
            ax_inset.plot(time_steps, normalized_amplitude_pred, linestyle='--', color=colours[1,:], linewidth=linewidth)
            ax_inset.set_xlim([16000, 20000]) 
            ax_inset.set_ylim([0, 0.8])  
            ax_inset.grid(False)
            ax_inset.tick_params(axis='x', labelsize=tick_fontsize - 2)
            ax_inset.tick_params(axis='y', labelsize=tick_fontsize - 2)

            # Highlight the zoomed area with a rectangle
            highlight = ax.axvspan(16000, 20000, color='gray', alpha=0.3)

            # Connect the inset plot to the highlighted area
            mark_inset(ax, ax_inset, loc1=2, loc2=4, fc="none", ec="0.5")


            plt.tight_layout()

            # Save the normalized received power plot as PNG and EPS files
            if savefig == 1:
                plt.savefig(f'2 users/full_cascaded_channel_user_{user}_30_dB_4QAM.png', format='png')
                plt.savefig(f'2 users/full_cascaded_channel_user_{user}_30_dB_4QAM.eps', format='eps')

            plt.show()

            # Plot the phase for each user
            fig, ax = plt.subplots(figsize=(12, 5))  # Size in inches
            ax.plot(time_steps, phase_test, label=f'Target', color=colours[0,:], linewidth=linewidth)
            ax.plot(time_steps, scaled_phase_pred, label=f'Predicted', linestyle='--', color=colours[1,:], linewidth=linewidth)
            ax.set_xlabel('Time (in samples)', fontname=fontname, fontsize=fontsize)
            ax.set_ylabel(f'Cascaded channel phase (in rad)', fontname=fontname, fontsize=fontsize)
            ax.set_xlim([0, 20000])
            #ax.ylim([0,0.8])
            legend_prop = FontProperties(family=fontname, size=legend_fontsize)
            ax.legend(frameon=False, prop=legend_prop)
            ax.grid(False)
            ax.tick_params(axis='x', labelsize=tick_fontsize)
            ax.tick_params(axis='y', labelsize=tick_fontsize)

            # Add vertical line at x=14000
            ax.axvline(x=15996, color='black', linewidth=2)

            # Add annotations for "Train" and "Test"
            ax.annotate('', xy=(14000, 0.3), xytext=(16000, 0.3), fontsize=fontsize,
                        arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=8), ha='center', va='center')
            ax.annotate('', xy=(17000, 0.3), xytext=(16000, 0.3), fontsize=fontsize,
                        arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=8), ha='center', va='center')
            
            # Add annotations for text
            ax.text(14000, 0.33, 'Train', fontsize=fontsize, ha='center', va='center')
            ax.text(17500, 0.33, 'Test', fontsize=fontsize, ha='center', va='center')

            ax.axvspan(16000, 20000, color='gray', alpha=0.3)

            # Save the phase plot as PNG and EPS files
            if savefig == 1:
                plt.savefig(f'2 users/full_phase_user_{user}_30_dB_4QAM.png', format='png')
                plt.savefig(f'2 users/full_phase_user_{user}_30_dB_4QAM.eps', format='eps')
                
            plt.show()

    def plot_model_evaluation_test(self):
        # Configuration settings
        linewidth = 2
        markersize = 10
        fontname = 'Times New Roman'
        fontsize = 20
        tick_fontsize = 14
        legend_fontsize = 14
        savefig = 0
        colours = np.array([
            [0.0000, 0.4470, 0.7410],
            [0.8500, 0.3250, 0.0980],
            [0.9290, 0.6940, 0.1250],
            [0.4940, 0.1840, 0.5560],
            [0.4660, 0.6740, 0.1880],
            [0.3010, 0.7450, 0.9330],
            [0.6350, 0.0780, 0.1840],
            [0.0000, 0.0000, 0.0000]
        ])
        
        M = self.y_test.shape[2]  # Number of users
        K = self.y_test.shape[0]
        # 55986,55986+23994
        # 63984, 63984+15996
        time_steps = np.arange(15984, 15984+3996)

        for user in range(M):
            # Extract the amplitude and phase of the received signals for each user
            amplitude_test = self.y_test[:, :, user, 0]
            phase_test = self.y_test[:, :, user, 1]
            amplitude_pred = self.y_pred[:, :, user, 0]
            phase_pred = self.y_pred[:, :, user, 1]

            # Calculate the mean of phase_pred and scale the deviations from the mean
            mean_phase_pred = np.mean(phase_pred)
            scaled_phase_pred = mean_phase_pred + 2.0 * (phase_pred - mean_phase_pred)

            # Calculate the amplitude of the received signals for each user
            amplitude_test = amplitude_test ** 2
            amplitude_pred = amplitude_pred ** 2

            # Normalize the amplitudes
            normalized_amplitude_test = amplitude_test
            normalized_amplitude_pred = amplitude_pred

            # Plot the normalized received power for each user and SNR
            fig, ax = plt.subplots(figsize=(12, 5))  # Size in inches
            ax.plot(time_steps, normalized_amplitude_test, label='Target', color=colours[0,:], linewidth=linewidth)
            ax.plot(time_steps, normalized_amplitude_pred, label='Predicted', linestyle='--', color=colours[1,:], linewidth=linewidth)
            ax.set_xlabel('Time (in samples)', fontname=fontname, fontsize=fontsize)
            ax.set_ylabel('Cascaded channel magnitude', fontname=fontname, fontsize=fontsize)
            #ax.set_xlim([63000, 80000])
            #ax.set_ylim([0, 0.2])
            legend_prop = FontProperties(family=fontname, size=legend_fontsize)
            ax.legend(frameon=False, prop=legend_prop)
            ax.grid(False)
            ax.tick_params(axis='x', labelsize=tick_fontsize)
            ax.tick_params(axis='y', labelsize=tick_fontsize)

            # Save the normalized received power plot as PNG and EPS files
            if savefig == 1:
                plt.savefig(f'1 user/full_cascaded_channel_1_user_-10_dB_16QAM.png', format='png')
                plt.savefig(f'1 user/full_cascaded_channel_1_user_-10_dB_16QAM.eps', format='eps')

            plt.show()

            # Plot the phase for each user
            plt.figure(figsize=(12, 5))  # Size in inches
            plt.plot(time_steps, phase_test, label=f'Target', color=colours[0,:], linewidth=linewidth)
            plt.plot(time_steps, scaled_phase_pred, label=f'Predicted', linestyle='--', color=colours[1,:], linewidth=linewidth)
            plt.xlabel('Time (in samples)', fontname=fontname, fontsize=fontsize)
            plt.ylabel(f'Cascaded channel phase', fontname=fontname, fontsize=fontsize)
            #plt.xlim([64000, 80000])
            #plt.ylim([0,0.8])
            legend_prop = FontProperties(family=fontname, size=legend_fontsize)
            plt.legend(frameon=False, prop=legend_prop)
            plt.grid(False)
            plt.xticks(fontname=fontname, fontsize=tick_fontsize)
            plt.yticks(fontname=fontname, fontsize=tick_fontsize)
            plt.tight_layout()

            # Save the phase plot as PNG and EPS files
            if savefig == 1:
                plt.savefig(f'1 user/full_phase_1_user_-10_dB_16QAM.png', format='png')
                plt.savefig(f'1 user/full_phase_1_user_-10_dB_16QAM.eps', format='eps')
                
            plt.show()

    # def plot_training_history(self, history):
    #     savefig =0
    #     plt.figure(figsize=(12, 6))
    #     plt.plot(history.history['loss'], label='Training Loss')
    #     #plt.plot(history['accuracy'], label='Training Accuracy')
    #     plt.xlabel('Epochs')
    #     plt.ylabel('Loss')
    #     plt.legend(["Loss"], loc="best")
    #     plt.grid(False)
    #     if savefig == 1:
    #             plt.savefig(f'test_history_2users_dl_4QAM_40SNR_1000epochs.png', format='png')
    #             plt.savefig(f'test_history_2users_dl_4QAM_40SNR_1000epochs.eps', format='eps')
    #     plt.show()
    
    # def plot_training_accuracy_history(self, history):
    #     plt.figure(figsize=(12, 6))
    #     plt.plot(history.history['accuracy'])
    #     plt.plot(history.history['val_accuracy'])
    #     plt.xlabel('Epochs')
    #     plt.ylabel('Accuracy')
    #     plt.title('Training and Validation Accuracy Over Epochs')
    #     plt.legend(["train", "val"], loc="best")
    #     plt.grid(False)
    #     plt.show()

    def plot_model_metrics(self):
        savefig = 0
        # Convert amplitude and phase to complex numbers
        y_test_complex = self.y_test[:, 0, :, 0] * np.exp(1j * self.y_test[:, 0, :, 1])
        y_pred_complex = self.y_pred[:, 0, :, 0] * np.exp(1j * self.y_pred[:, 0, :, 1])
        
        # Flatten the arrays
        y_test_flat = y_test_complex.flatten()
        y_pred_flat = y_pred_complex.flatten()
        
        evaluation = Evaluation(self.y_test, self.y_pred)
        # Calculate metrics
        nrmse_val = evaluation.nrmse()
        mase_val = evaluation.mase()
        mape_val = evaluation.mape()
        #r2_val = r2_score(np.abs(y_test_flat), np.abs(y_pred_flat))
        r2_val = evaluation.r2_score()
        rmse_val = evaluation.rmse()
        mae_val = evaluation.mae()

        print(f"NRMSE: {nrmse_val}")
        print(f"MASE: {mase_val}")
        print(f"MAPE: {mape_val}")
        print(f"R2 Score: {r2_val}")
        print(f"RMSE: {rmse_val}")
        print(f"MAE: {mae_val}")
        

        # Metrics for plotting
        metrics = ['NRMSE', 'MASE', 'MAPE', 'R2 Score', 'RMSE', 'MAE']
        values = [nrmse_val, mase_val, mape_val, r2_val, rmse_val, mae_val]

        # Plotting
        plt.figure(figsize=(10, 6))
        plt.bar(metrics, values, color='skyblue')
        plt.xlabel('Metrics')
        plt.ylabel('Values')
        #plt.title('Model Evaluation Metrics')
        plt.ylim([0, max(values) + 0.05 * max(values)])  # Adjust y-axis limit
        # Annotate each bar with the value
        for i, v in enumerate(values):
            plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom')

        if savefig == 1:
                plt.savefig(f'1 user/full_model_metrics_1_user_-10_dB_16QAM.png', format='png')
                plt.savefig(f'1 user/full_model_metrics_1_user_-10_dB_16QAM.eps', format='eps')
        plt.show()


In [ ]:
# ## Plots
results_2_user = scipy.io.loadmat('./new_model/results_2_users_dl_4QAM_7.mat')
y_pred_all_2 = results_2_user['y_pred_all']
y_true_all_2 = results_2_user['y_true_all'] 
X_all_2 = results_2_user['X_all'] 
y_pred_test_2 = results_2_user['y_pred_test']  
y_pred_train_save_2 = results_2_user['y_pred_train']  
y_true_test_2 = results_2_user['y_true_test']  
y_true_train_2 = results_2_user['y_true_train']  
X_test_save_2 = results_2_user['X_test']  
X_train_save_2 = results_2_user['X_train']  
# results_1_user['SNR_dB'] SNR
# results_1_user['Q'] Q


In [ ]:
# Load the JSON file containing the training history
with open('./new_model/history_per_snr_5.json', 'r') as json_file:
    results = json.load(json_file)

# Define font properties
fontname = 'Times New Roman'
fontsize = 20
legendsize = 18
linewidth = 2
markersize = 10

# Set global font properties
plt.rc('font', family=fontname, size=fontsize)

# Extract the loss values for each SNR
history_per_snr = results

# Define the SNR values to be plotted
selected_snr_values = ['SNR_30']

# Create a plot for the loss over epochs for the selected SNR values
plt.figure(figsize=(10, 7.5))  # Adjusted to 800x600 in pixel terms assuming 100 DPI
for snr, history in history_per_snr.items():
    if snr in selected_snr_values:
        plt.plot(history['loss'], label=f'{snr.replace("SNR_", "")} dB', linewidth=linewidth)

# Customize axes
plt.xlabel('Epochs', fontsize=fontsize, fontname=fontname)
plt.ylabel('Loss', fontsize=fontsize, fontname=fontname)
plt.ylim([0.0060, 0.02300])
plt.xlim([-0.1, 100])

# Customizing legend
legend_prop = FontProperties(family=fontname, size=legendsize)
legend = plt.legend(frameon=False, prop=legend_prop)
for line in legend.get_lines():
    line.set_linewidth(2.0)  # Increase width of legend lines

# Customize grid and borders
plt.grid(False)
plt.gca().spines['top'].set_linewidth(1.1)
plt.gca().spines['right'].set_linewidth(1.1)
plt.gca().spines['left'].set_linewidth(1.1)
plt.gca().spines['bottom'].set_linewidth(1.1)

plt.savefig(f'new_model_loss_100epochs.png', format='png')
plt.savefig(f'new_model_loss_100epochs.eps', format='eps')

plt.show()

In [ ]:
plotter = GeneratePlots(y_true_all_2[:,:,:,:,8], y_pred_all_2[:,:,:,:,8])
plotter.plot_model_evaluation()

In [ ]:
plotter = GeneratePlots(y_true_test_2[:,:,:,:,8], y_pred_test_2[:,:,:,:,8])
plotter.plot_model_evaluation_test()
plotter.plot_model_metrics()

In [ ]:
# List of filenames to load manually
filenames = [
    'inference_times_per_snr_8.json'
    # Add more filenames as needed
]

# Path to the directory containing your files
directory_path = './new_model/'

# List to store the results
results = []

# Load each file and calculate the average
for filename in filenames:
    file_path = f"{directory_path}{filename}"
    
    # Open and load the JSON data
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    # Calculate the average inference time
    average_time = sum(data.values()) / len(data)
    
    # Convert to milliseconds (if times are in seconds)
    average_time_ms = average_time * 1000
    
    # Append the result
    results.append((filename, average_time_ms))

# Print the results
for filename, avg_time in results:
    print(f'{filename}, Average Inference Time (ms): {avg_time:.6f}')